<!-- ---
title: How to train a multi output model 
weight: 12
downloads: true
sidebar: true
summary: This example demonstrates how you can train a multi output model, evaluate it and calculate relevant Ignite metrics using utility functions and output transforms.
tags:
  - load checkpoint
--- -->
# How to train a multi output model 

## Required Dependencies

In [32]:
!pip install pytorch-ignite -q

## Imports

In [33]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import MeanAbsoluteError

## Data

For out dataset, we will create a custom dataset which takes a range and calculates two labels via the `power2()` and `mul2()` methods.

In [34]:
def power2(x):
    return x**2

def mul2(x):
    return 2*x

class RangeDataset(Dataset):
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def __len__(self):
        return self.end - self.start + 1

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        item = self.start + idx
        y1 = power2(item)
        y2 = mul2(item)
        sample = [torch.Tensor([item]), torch.Tensor([y1]), torch.Tensor([y2])]
        return sample

train_dataset = RangeDataset(1, 80)
val_dataset = RangeDataset(90, 100)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

## Model

Let's create a dummy `Net()`. We have two linear layers for predicting the two labels which we can simply return from `forward()`.

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        self.model1 = nn.Linear(1, 1)
        self.model2 = nn.Linear(1, 1)

    def forward(self, x):
        return self.model1(x), self.model2(x)

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.MSELoss()

## Trainer

Now we will create a custom `trainer` which unpacks a batch, makes a prediction, calculates the two losses and returns their sum.

In [42]:
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y1, y2 = batch[0].to(device), batch[1].to(device), batch[2].to(device)
    y_pred1, y_pred2 = model(x)
    loss1 = criterion(y_pred1, y1)
    loss2 = criterion(y_pred2, y2)

    loss1.backward()
    loss2.backward()
    optimizer.step()
    return loss1.item() + loss2.item()

trainer = Engine(train_step)

## Evaluator

For the evaluation step, we will leverage [`create_supervised_evaluator()`](https://pytorch.org/ignite/generated/ignite.engine.create_supervised_evaluator.html#create-supervised-evaluator) to adapt to our use case. 

At first, we need to prepare our batch since we have two labels. The `prepare_batch()` method receives the necessary parameters and must return a tuple of inputs and targets `(x, y)`. Here we have adapted `y` to be another tuple containing the two labels.

In [43]:
def prepare_batch(batch, device, non_blocking):
    x, y1, y2 = batch[0].to(device), batch[1].to(device), batch[2].to(device)
    return (x, (y1, y2))

val_evaluator = create_supervised_evaluator(model, prepare_batch=prepare_batch, device=device)

The above evaluator translates into:
```python
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = prepare_batch(...)
        y_pred = model(x)
        return y_pred, y

val_evaluator = Engine(validation_step)
```

You can also pass an `output_transform()` to `val_evaluator` for more complex needs as follows:
```python
def output_transform(x, y, y_pred):
    return {'y_pred': y_pred, 'y': y}
```

## Metrics

Since our dummy dataset is better suited for a regression task, let's use [`MeanAbsoluteError()`](https://pytorch.org/ignite/generated/ignite.metrics.MeanAbsoluteError.html#meanabsoluteerror) as our metric. However, we cannnot directly use it and need to pass a transform method to further preprocess the predictions and labels returned. Below, we are simply adding the predictions and labels respectively to return our final `y_pred` and `y` upon which the mtric will be calculated.

In [44]:
def mae_output_transform(output):
    y_pred1, y_pred2 = output[0]
    y1, y2 = output[1]
    y_pred = y_pred1 + y_pred2
    y = y1 + y2
    return y_pred, y

val_metrics = {
    'mae': MeanAbsoluteError(mae_output_transform),
}

for name, metric in val_metrics.items():
    metric.attach(val_evaluator, name)

Finally, let's log our results and start training.

In [45]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg MAE: {metrics['mae']:.2f}")

In [46]:
trainer.run(train_loader, max_epochs=1)

Validation Results - Epoch[1] Avg MAE: 9322.08


State:
	iteration: 20
	epoch: 1
	epoch_length: 20
	max_epochs: 1
	output: 10597183.16796875
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

Although, the MAE for our dummy model is really large, this code can easily be adapted to fit multioutput models and then calculate respective metrics!